In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
sys.argv=[''] 
del sys

In [3]:
cd "/root/SymmNeRF/"

/root/SymmNeRF


In [4]:
from opt import config_parser
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import setproctitle
import tqdm 
parser = config_parser()
args = parser.parse_args()

device = torch.device("cuda:0")
det = args.det
lindisp = args.lindisp
args.distributed = False
args.local_feature_ch = 512
args.local_rank = 0
args.eval_scene = 'cars'
args.eval_dataset = 'srns_dataset'

args.no_load_opt = True
args.no_load_scheduler = True

setproctitle.setproctitle('[Gaetan - SymmNeRF Test]')

In [5]:
# Create training dataset
from torch.utils.data import DataLoader,Subset
from datasets import dataset_dict, create_training_dataset
import matplotlib.pyplot as plt 


test_dataset = dataset_dict[args.eval_dataset](args,'test',args.eval_scene)

test_idx = np.random.randint(1,len(test_dataset),100)
test_subset = Subset(test_dataset,test_idx)
test_loader = DataLoader(test_subset,batch_size = 1,shuffle = True)

SRNsDataset:  /data/srn_cars/cars_test


In [6]:
from model.model import HyperNeRFResNetSymmLocal

### Original SymmNeRF with our modifications.

In [7]:
args.enforce_symmetry = {'status': True, 'on_coarse': True,'on_fine': True,'concatenate_on_coarse':True, 'concatenate_on_fine':True}
args.feature = {'F':{'use_CNN':True,'use_VIT':False},'z':{'use_CNN':True,'use_VIT':False}}
args.with_vit = False
args.archCNN = 'resnet34'
args.local_feature_ch = 512
args.num_local_layers = 2

model_enforce_symm_with_concat = HyperNeRFResNetSymmLocal(args,ckpts_folder = '/root/SymmNeRF/logs/enforceSymmFineOnlyWithConcat/ckpts')

[Info] Reloading from /root/SymmNeRF/logs/enforceSymmFineOnlyWithConcat/ckpts/model_400000.pth, starting at step=400000


### What we would like to train. 

In [24]:
args.enforce_symmetry = {'status': True, 'on_coarse': True,'on_fine': True,'concatenate_on_coarse':True, 'concatenate_on_fine':True}
args.feature = {'F':{'use_CNN':True,'use_VIT':False},'z':{'use_CNN':True,'use_VIT':False}}
args.with_vit = False
args.archCNN = 'resnet-vision'
args.local_feature_ch = 128
args.latent_dim = 256
args.num_local_layers = 2

model = HyperNeRFResNetSymmLocal(args,ckpts_folder = '')

[Info] No ckpts found, training from scratch...


### Test the implementation

In [26]:
x = torch.zeros([1,3,128,128]).to(device)

z,Fcnn,Fvit = model.encode(x)

print(z.shape)
print(Fcnn.shape)

torch.Size([1, 256])
torch.Size([1, 64, 64, 64])


In [8]:
from model.sample_ray import RaySampler
from model.render_ray import * 
from model.projection import Projector
import numpy as np 

In [9]:
# Data to overfit on.
it = iter(test_loader)
test_data = next(it)

# Load training rays
ray_sampler = RaySampler(test_data)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [10]:
# Symmetry matrix. 
M = torch.tensor([[-1.,0.,0.],[0.,1.,0.],[0.,0.,1.]]).to(device)

# Projector 
projector = Projector(device = device)

# Render view. 
render_view = np.random.randint(0,250,1)[0]

In [11]:
# Prediction Image.
Ipred = get_intermediate_visuals(args,model_enforce_symm_with_concat,projector,M,device,ray_sampler,render_view)

# Target Image. 
It = ray_sampler.render_imgs[0][render_view].permute(1, 2, 0).cpu()
It_np = It.numpy()
# Source Image. 
Is = ray_sampler.render_imgs[0][ray_sampler.src_view[0]].permute(1,2,0)
Is_np = Is.numpy()

In [23]:
from model.feature_network import * 

In [27]:
model = ResNetVision(128)
x = torch.zeros([1,3,128,128])
z = model(x) 
F = model.latent
print(F.shape)
print(y.shape)

torch.Size([1, 64, 64, 64])
torch.Size([1, 256])


In [19]:
import torch.nn as nn
avg_pool = nn.AdaptiveAvgPool2d(1)
y2 = avg_pool(y).squeeze(-1).squeeze(-1)
print(y2.shape)

torch.Size([1, 64])


In [22]:
features = 128
z_latent_mlp = nn.Sequential(nn.Linear(features//2,features),
                                          nn.Linear(features,2*features)
                                         )

z = z_latent_mlp(y2)
print(z.shape)

torch.Size([1, 256])
